# ENSF 519.01 Applied Data Science 
**Final Project - Prediciting Alberta Natural Gas Prices using Machine Learning**

**Due:** Sometime in December


**Team Members:**
James Bews,
Kendal Blais,
Tevin Schmidt,
Peter Schulze,

**Phase I:** Collect, Integrate and Clean data


**Data Collection**

In [1]:
import pandas as pd

# aggregate consumption
cad_data = pd.read_csv('CAD_NatGasSupply_Monthly.csv', thousands=',').rename(columns={'Month': 'Date'}).set_index('Date') # volumes in cubic meters
us_prod_data = pd.read_csv('US_NatGasProd_Monthly.csv').set_index('Date') # volumes in MMcf
us_demand_data = pd.read_csv('US_NatGasDemand_Monthly.csv').set_index('Date').dropna() # volumes in MMcf
price_data = pd.read_csv('US_CAN_NatGas_Prices.csv').set_index('Date') # prices in USD


In [5]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

# Due to 

storage_stats_url = 'https://www150.statcan.gc.ca/t1/tbl1/en/cv!recreate.action?pid=2510005701&selectedNodeIds=2D2,3D1&checkedLevels=0D1&refPeriods=20160101,20190801&dimensionLayouts=layout2,layout3,layout2,layout2&vectorDisplay=false'

def get_page_html(url):
    return requests.get(url,timeout=10)

storage_stats_html = get_page_html(storage_stats_url).content
storage_stats_html

b'<html><head><meta http-equiv="Pragma" content="no-cache"/>\r\n<meta http-equiv="Expires" content="-1"/>\r\n<meta http-equiv="CacheControl" content="no-cache"/>\r\n<script>\r\n(function(){})();</script><script type="text/javascript">\r\nfunction decode_string(in_str) { return decodeURIComponent(in_str); }\r\nfunction decode_action() {var f = document.forms[0];if (f.attributes[\'action\'] != undefined) {f.attributes[\'action\'].value = decode_string(f.attributes[\'action\'].value);} else { f.action = decode_string(f.action);}}function submit_form() {var e = document.forms[0].elements;e[1].value = decode_string(e[1].value);e[2].value = decode_string(e[2].value);e[5].value = decode_string(e[5].value);e[7].value = decode_string(e[7].value);document.forms[0].submit();}function cookie_redirect() {var cookie = \'\';var e = document.forms[0].elements;var uri = (document.forms[0].attributes[\'action\'] != undefined) ?document.forms[0].attributes[\'action\'].value : document.forms[0].action;var

In [6]:

parser = BeautifulSoup(storage_stats_html, 'html.parser')

column_name_selector = '.pub-table .uom-center.bold'
date_column = list(map(lambda elem: elem.text, parser.select(column_name_selector)))[2:]
date_column = list(map(lambda elem: datetime.strptime(elem, '%B %Y').strftime('%b-%y'), date_column))

column_name_selector = '.pub-table .highlight-row .align-right'
closing_inventory_column = list(map(lambda elem: float(elem.text.replace(',', '')), parser.select(column_name_selector)))

storage_stats_dict = {'Date': date_column, 'CAD Storage Volumes (MMcf)': closing_inventory_column}
storage_stats_df = pd.DataFrame(storage_stats_dict).set_index('Date')
# storage_stats_df contains closing inventory of a given month in cubic meters
storage_stats_df


,CAD Storage Volumes (MMcf)
Date,


**Data Preparation**

- Canadian demand need to be agregated
- Volumes must be converted to MMcf
- USD must be convetred to CAD


In [ ]:
# Aggregate CAD Consumption
cad_data['CAD Natural Gas Consumption (MMcf)'] = cad_data['Residential consumption']+cad_data['Commercial consumption']+cad_data['Industrial consumption']
cad_data = cad_data.rename(columns={'Marketable production': 'CAD Natural Gas Production (MMcf)'})
cad_data = cad_data.drop(['Gross withdrawals','Imports','Residential consumption', 'Industrial consumption', 'Commercial consumption', 'Exports', 'Opening inventory', 'Closing inventory', 'Inventory change'], axis=1)
cad_data

In [ ]:
# Convert volumes 

# cubic meters to cubic feet -> 35.3146667
# divide by 1 million to convert cubic feet to MMcf
# times by 1000 because the Canadian data was given to us with a scale of 1000 
cm_to_mmcf = 35.3146667 / 1000000 * 1000
# First convert storage_stats_df from cubic meters to MMcf
conv_cad_storage = storage_stats_df['CAD Storage Volumes (MMcf)'].apply(lambda x: x * cm_to_mmcf)

# Second convert price_data from USD to CAD
conv_price_data = pd.DataFrame()
conv_price_data['AECO (CAD/MMBtu)'] = price_data.apply(
    lambda x: x['AECO (USD/MMBtu)'] / x['CADUSD'], axis=1)
conv_price_data['AECO (CAD/GJ)'] = conv_price_data.apply(
    lambda x: round(x['AECO (CAD/MMBtu)'] * 1.055056, 2), axis=1)
conv_price_data = conv_price_data.drop(['AECO (CAD/MMBtu)'], axis=1)
# Third convert cad_data from cubic meters to MMcf
conv_cad_data = cad_data.apply(lambda x: x * cm_to_mmcf)
conv_price_data

**Data Exploration**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#CAD Production
ax = plt.gca()
conv_cad_data.plot(kind='line', y='CAD Natural Gas Production (MMcf)', ax=ax)
plt.show()

In [ ]:
#CAD Consumption
ax = plt.gca()
conv_cad_data.plot(kind='line', y='CAD Natural Gas Consumption (MMcf)', ax=ax)
plt.show()

In [ ]:
# CAD Storage Volumes
ax = plt.gca()
storage_stats_df.plot(kind='line', y='CAD Storage Volumes (MMcf)', ax=ax)
plt.show()

In [ ]:
#US Production
ax = plt.gca()
us_prod_data.plot(kind='line', y='U.S. Dry Natural Gas Production (MMcf)', ax=ax)
plt.show()

In [ ]:
#US Demand
ax = plt.gca()
us_demand_data.plot(kind='line', y='U.S. Natural Gas Total Consumption (MMcf)', ax=ax)
plt.show()

In [ ]:
# AECO Nat Gas Pricing
ax = plt.gca()
conv_price_data.plot(kind='line', y='AECO (CAD/GJ)', ax=ax)
plt.show()

In [ ]:
#Merging all data into one dataframe

total_us_data = us_prod_data.merge(us_demand_data, left_index=True, right_on='Date')
total_cad_data = conv_cad_data.merge(conv_cad_storage, left_index=True, right_on='Date')

total_data = total_us_data.merge(total_cad_data, left_index=True, right_on='Date')
total_data = total_data.apply(lambda x: round(x.apply(lambda y: int(y))))
features = total_data.merge(conv_price_data, left_index=True, right_on='Date')
features = features.reset_index()
features['Date'] = pd.to_datetime(features.Date, format='%b-%y')
features = features.set_index('Date')
features

**Data Prediction**

**Testing Causality (Granger's Causality Test)**

If the p-value between 2 variables X and Y is less than 0.05, then X causes Y

In [ ]:
import numpy as np
from statsmodels.tsa.stattools import grangercausalitytests

def grangers_test(data, variables, test='ssr_chi2test', maxlag=12, verbose=False):
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for column in df.columns:
        for row in df.index:
            test_result = grangercausalitytests(data[[row,column]], maxlag=maxlag, verbose=verbose)
            p_values = [round(test_result[i+1][0][test][1], 4) for i in range(maxlag)]
            
            min_p_value = np.min(p_values)
            df.loc[row,column] = min_p_value
            
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_test(features, variables=features.columns)

In [ ]:
n_obs = 10
X_train, X_test = features[0:-n_obs], features[-n_obs:]

print(X_train.shape)
print(X_test.shape)

In [ ]:
from statsmodels.tsa.stattools import adfuller
from scipy import stats

def adf_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1],4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue']
    def adjust(val, length=6): return str(val).ljust(length)
    
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Significance Level    = {signif}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")
    return output
          
def test_stationality(df):
    lags = []
    for name, column in df.iteritems():
          output = adf_test(column, name=column.name)
          lags.append(output['n_lags'])
          print('\n')
    lags = np.array(lags)
    return stats.mode(lags)[0][0]

        
lag = test_stationality(X_train)
print('Lag -> ', lag)

In [ ]:
#First Difference

differenced = X_train.diff().dropna()
lag = test_stationality(differenced)
print('Lag -> ', lag)
ax = plt.gca()
differenced.plot(kind='line', y='U.S. Dry Natural Gas Production (MMcf)', ax=ax)
plt.show()

In [ ]:
differenced = differenced.diff().dropna()
test_stationality(differenced)
print('Lag -> ', lag)
ax = plt.gca()
differenced.plot(kind='line', y='U.S. Dry Natural Gas Production (MMcf)', ax=ax)
plt.show()

In [ ]:
from statsmodels.tsa.api import VAR

model = VAR(differenced)
model_fitted = model.fit(lag)

In [ ]:
forecast_input = differenced.values[-lag:]
fc = model_fitted.forecast(y=forecast_input, steps=n_obs)
df_forecast = pd.DataFrame(fc, index=features.index[-n_obs:], columns=features.columns + '_2d')
df_forecast

In [ ]:
# Undifference

columns = X_train.columns
for column in columns:
    df_forecast[str(column) + '_ld'] = (X_train[column].iloc[-1] - X_train[column].iloc[-2]) + df_forecast[str(column) + '_2d'].cumsum()
    # Roll back 1st diff 
    df_forecast[str(column) + '_fc'] = X_train[column].iloc[-1] + df_forecast[str(column) + '_ld'].cumsum()

df_forecast.loc[:,['U.S. Dry Natural Gas Production (MMcf)_fc', 'U.S. Natural Gas Total Consumption (MMcf)_fc', 'CAD Natural Gas Production (MMcf)_fc', 'CAD Natural Gas Consumption (MMcf)_fc', 'CAD Storage Volumes (MMcf)_fc', 'AECO (CAD/GJ)_fc']]


In [ ]:
fig, axes = plt.subplots(nrows=int(len(features.columns)/2), ncols=2, dpi=150, figsize=(10,10))
for i, (col,ax) in enumerate(zip(features.columns, axes.flatten())):
    df_forecast[col+'_fc'].plot(legend=True, color='b', ax=ax).autoscale(axis='x',tight=True)
    X_test[col][-n_obs:].plot(legend=True, color='r', ax=ax);
    ax.set_title(col + ": Forecast vs Actuals")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();